# hello world

In [1]:
data class User(val name: String, val age:Int)

In [2]:
val user1 = User(name="hogeta", age=10)
user1

User(name=hogeta, age=10)

# Install artifacts

In [3]:
@file:Repository("https://repo.maven.org/maven2")

In [4]:
@file:DependsOn("org.nd4j:nd4j-api:0.9.1")
@file:DependsOn("org.nd4j:nd4j-native:0.9.1")
@file:DependsOn("org.nd4j:nd4j-native-platform:0.9.1")

In [5]:
import org.nd4j.linalg.factory.Nd4j

In [6]:
val arr = Nd4j.create(doubleArrayOf(1.0, 2.0, 3.0))
arr

[1.00,  2.00,  3.00]

In [7]:
:help

Commands:
    :help - display help
    :classpath - show current classpath
    :vars - get visible variables values

Magics
    %use - injects code for supported libraries: artifact resolution, default imports, initialization code, type renderers
        Usage: %use klaxon(5.5), lets-plot
    %trackClasspath - logs any changes of current classpath. Useful for debugging artifact resolution failures
        Usage: %trackClasspath [on|off]
    %trackExecution - logs pieces of code that are going to be executed. Useful for debugging of libraries support
        Usage: %trackExecution [all|generated|off]
    %useLatestDescriptors - use latest versions of library descriptors available. By default, bundled descriptors are used
        Usage: %useLatestDescriptors [on|off]
    %output - output capturing settings
        Usage: %output --max-cell-size=1000 --no-stdout --max-time=100 --max-buffer=400
    %logLevel - set logging level
        Usage: %logLevel [off|error|warn|info|debug]

Supporte

# Titanic

In [8]:
%use dataframe

In [9]:
val df = DataFrame.read("./data/train.csv")
df.head()

DataFrame: rowsCount = 5, columnsCount = 12

In [10]:
val trainDf = DataFrame.read("./data/train.csv")
val testDf = DataFrame.read("./data/test.csv")
val dataDf = trainDf.concat(testDf)

In [11]:
trainDf.describe()
val trainSize = trainDf.rowsCount()

In [12]:
trainSize

891

In [13]:
testDf.describe()

DataFrame: rowsCount = 11, columnsCount = 12

In [14]:
println("train:${trainDf.size()}, test:${testDf.size()}, data:${dataDf.size()}")

train:891 x 12, test:418 x 11, data:1309 x 12


In [15]:
dataDf.describe()

DataFrame: rowsCount = 12, columnsCount = 12

In [16]:
dataDf.columnNames()

[PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked]

In [17]:
dataDf.Age

... showing only top 20 of 1309 rows DataColumn: name = "Age", type = Double?, size = 1309

In [18]:
val ageAvg = dataDf.Age.mean()
val ageStd = dataDf.Age.std()
println("${ageAvg},${ageStd}")

29.881137667304014,14.413493211271334


In [19]:
import kotlin.random.Random

// テキトーな乱数で補完する
var filledDataDf = dataDf.fillNA{Age}.withValue((Random.nextDouble((ageAvg - ageStd), (ageAvg + ageStd))).toInt().toDouble())
filledDataDf = filledDataDf.fillNA{Embarked}.withValue("S")
filledDataDf = filledDataDf.convert{Embarked}.with { e -> when(e) {
    "S" -> 0
    "C" -> 1
    "Q" -> 2
    else -> throw Exception()
}}
filledDataDf = filledDataDf.convert {Sex}.with { s -> when(s) {
    "male" -> 0
    "female" -> 1
    else -> 2
}}

In [20]:
filledDataDf = filledDataDf.remove {Name and PassengerId and SibSp and Parch and Ticket and Cabin}

In [21]:
filledDataDf.head(10)

DataFrame: rowsCount = 10, columnsCount = 6

In [22]:
%use lets-plot

In [23]:
val p = letsPlot(df.toMap()){x="Age"}
p + geomHistogram(binWidth=5, color="black", fill="white") { y = "..density.." } +
    geomDensity(alpha=0.2, fill=0xFF6666)

In [24]:
val p = letsPlot(df.toMap()){x="Sex"; y="Age"}
p + geomBoxplot(outlierColor="red", outlierShape=8, outlierSize=5){fill="Sex"}

In [25]:
val batchSize = 891
var trainDf = filledDataDf.head(batchSize)
var testDf = filledDataDf.tail(filledDataDf.rowsCount() - batchSize)

In [26]:
testDf = testDf.remove { Survived }

In [27]:
trainDf.writeCSV("./data/shaped_train.csv")
testDf.writeCSV("./data/shaped_test.csv")

In [28]:
@file:DependsOn("org.jetbrains.kotlinx:kotlin-deeplearning-api:0.4.0")

In [29]:
import org.jetbrains.kotlinx.dataframe.DataFrame
import org.jetbrains.kotlinx.dataframe.api.remove
import org.jetbrains.kotlinx.dataframe.api.rows
import org.jetbrains.kotlinx.dl.api.core.Sequential
import org.jetbrains.kotlinx.dl.api.core.WritingMode
import org.jetbrains.kotlinx.dl.api.core.layer.core.Input
import org.jetbrains.kotlinx.dl.api.core.layer.core.Dense
import org.jetbrains.kotlinx.dl.api.core.loss.Losses
import org.jetbrains.kotlinx.dl.api.core.metric.Metrics
import org.jetbrains.kotlinx.dl.api.core.optimizer.Adam
import org.jetbrains.kotlinx.dl.api.core.summary.printSummary
import org.jetbrains.kotlinx.dl.dataset.OnHeapDataset
import org.jetbrains.kotlinx.dl.dataset.fashionMnist
import java.io.File

In [30]:
fun <T> DataFrame<T>.toOnHeapDataset(labelColumnName: String): OnHeapDataset {
    return OnHeapDataset.create(
        dataframe = this,
        yColumn = labelColumnName
    )
}

fun OnHeapDataset.Companion.create(
    dataframe: DataFrame<Any?>,
    yColumn: String
): OnHeapDataset {
    fun extractX(): Array<FloatArray> =
        dataframe.remove(yColumn).rows()
            .map { (it.values() as List<Float>).toFloatArray() }.toTypedArray()

    fun extractY(): FloatArray =
        dataframe.get { yColumn<Float>() }.toList().toFloatArray()

    return create(
        ::extractX,
        ::extractY
    )
}

In [31]:
val (train, test) = trainDf.toOnHeapDataset("Survived").split(0.9)

In [36]:
val model = Sequential.of(
    Input(5),
    Dense(2)
)
model.use {
    it.compile(
        optimizer = Adam(),
        loss = Losses.BINARY_CROSSENTROPY,
        metric = Metrics.ACCURACY
    )
    it.printSummary()
    it.fit(
        dataset = train,
        epochs = 10,
        batchSize = batchSize
    )
    val accuracy = it.evaluate(dataset = test, batchSize = test.xSize()).metrics[Metrics.ACCURACY]
    println("Accuracy: $accuracy")
    it.save(File("model/my_model"), writingMode = WritingMode.OVERRIDE)
}

Model type: Sequential
______________________________________________________________________________
Layer (type)                           Output Shape              Param #      
input_1(Input)                         [None, 5]                 0            
______________________________________________________________________________
dense_2(Dense)                         [None, 2]                 12           
______________________________________________________________________________
Total trainable params: 12
Total frozen params: 0
Total params: 12
Accuracy: 0.644444465637207


In [33]:
import org.jetbrains.kotlinx.dl.api.core.Sequential
import org.jetbrains.kotlinx.dl.api.core.activation.Activations
import org.jetbrains.kotlinx.dl.api.core.initializer.HeNormal
import org.jetbrains.kotlinx.dl.api.core.layer.core.Dense
import org.jetbrains.kotlinx.dl.api.core.layer.core.Input
import org.jetbrains.kotlinx.dl.api.core.loss.Losses
import org.jetbrains.kotlinx.dl.api.core.metric.Metrics
import org.jetbrains.kotlinx.dl.api.core.optimizer.Adam
import org.jetbrains.kotlinx.dl.dataset.fashionMnist

In [38]:
val model = Sequential.of(
    Input(5),
    Dense(
        outputSize = 9,
        activation = Activations.Relu,
        kernelInitializer = HeNormal(),
        biasInitializer = HeNormal()
    ),
    Dense(
        outputSize = 9,
        activation = Activations.Relu,
        kernelInitializer = HeNormal(),
        biasInitializer = HeNormal()
    ),
    Dense(
        outputSize = 2,
        activation = Activations.Sigmoid,
        kernelInitializer = HeNormal(),
        biasInitializer = HeNormal()
    )
)
model.use {
    it.compile(
        optimizer = Adam(),
        loss = Losses.BINARY_CROSSENTROPY,
        metric = Metrics.ACCURACY
    )
    it.printSummary()
    it.fit(
        dataset = train,
        epochs = 200,
        batchSize = 50
    )
    val accuracy = it.evaluate(dataset = test, batchSize = test.xSize()).metrics[Metrics.ACCURACY]
    println("Accuracy: $accuracy")
    it.save(File("model/my_model"), writingMode = WritingMode.OVERRIDE)
}


Model type: Sequential
______________________________________________________________________________
Layer (type)                           Output Shape              Param #      
input_1(Input)                         [None, 5]                 0            
______________________________________________________________________________
dense_2(Dense)                         [None, 9]                 54           
______________________________________________________________________________
dense_3(Dense)                         [None, 9]                 90           
______________________________________________________________________________
dense_4(Dense)                         [None, 2]                 20           
______________________________________________________________________________
Total trainable params: 164
Total frozen params: 0
Total params: 164
Accuracy: 0.8111110925674438
